# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
import pickle
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
import re

# Machine Learning
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
# # from sklearn.multiclass import OneVsRestClassifier
# # from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report
# from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponseTable.db')
df = pd.read_sql_table('DisasterResponseTable',engine)
# print(Y)

In [ ]:
df.describe()

In [ ]:
# Given value 2 in the related field are neglible so it could be error. Replacing 2 with 1 to consider it a valid response.
# Alternatively, we could have assumed it to be 0 also. In the absence of information I have gone with majority class.
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [ ]:
df['message'][0]

In [ ]:
df.head()

In [ ]:
# df.iloc[:,1:2]

In [ ]:
X = df['message'].values
y = df.iloc[:,4:]
category_names = y.columns
y = y.values
# y = y.astype('int')

In [ ]:
category_names

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
clf = MultiOutputClassifier(KNeighborsClassifier())
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])
# pipeline = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer()),
#         ('clf', MultiOutputClassifier(RandomForestClassifier()))
#     ])

# #optimize classifier with GridSearch
# parameters = {
#     'clf__estimator__n_estimators': [10, 50, 100],
#     'clf__estimator__min_samples_split': [2, 3] 
#     }
# model = GridSearchCV(pipeline, param_grid=parameters)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
type(X_train)

In [ ]:
type(y_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# y_pred = pipe.predict(x_test)
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
# target_names = categories.columns.values
# classification_report(y_test, y_pred, target_names = target_names, output_dict=True)
# print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

In [ ]:
print(classification_report(y_test, y_prediction_test, target_names=category_names))##y.columns.values))

In [ ]:
# Print classification report on training data
print(classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train.values, y_train.values)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

In [ ]:
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# !pip install gensim

In [ ]:
# !pip install dill

In [ ]:
# !pip3 install xgboost
# !conda install -c anaconda py-xgboost

In [ ]:
# import xgboost as xgb

In [ ]:
# pipeline_advanced = Pipeline([
#     ('vec',CountVectorizer(tokenizer=tokenize)),
#     ('tfidf',TfidfTransformer()),
#     ('clf',MultiOutputClassifier(RandomForestClassifier()))
# ])
# clf = MultiOutputClassifier(RandomForestClassifier(random_state=42))
# pipeline_advanced = Pipeline([
#         ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf_transformer', TfidfTransformer())
#             ]))
            
#         ])),

#         ('classifier', clf)
#     ])
# param_grid = { 
#     'n_estimators': [200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
#     'criterion' :['gini', 'entropy']
# }
# # parameters2 = {'bootstrap': [True, False],
# #                  'max_depth': [10, 20, 30, 40, 50, 60, 70],
# #                  'max_features': ['auto', 'sqrt'],
# #                  'min_samples_leaf': [1, 2, 4],
# #                  'min_samples_split': [2, 5],
# #                  'n_estimators': [50,100,150]}
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline2.fit(X_train.values, y_train.values)

In [ ]:
#Use pipeline2 which includes StartingVerbEstimator


y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

In [ ]:
# cv = GridSearchCV(pipeline_advanced, param_grid=param_grid, scoring='f1_micro', n_jobs=-1)
# cv.fit(X_train.values, y_train.values)

In [ ]:
#Use pipeline2 which includes StartingVerbEstimator
# X_train, X_test, y_train, y_test = train_test_split(X, y)
# pipeline_fitted = pipeline2.fit(X_train.values, y_train.values)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
m = pickle.dumps(cv,'classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.